In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mount at content/drive

In [83]:
import pandas as pd
import numpy as np
import os
import operator
import re
import sys

import seaborn as sns
from datetime import datetime
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


%matplotlib inline

In [84]:
df = pd.read_csv("/content/drive/MyDrive/The Eggcellent Predictor /Data/eggprice2.csv")

In [85]:
df.head(3)

,Program,Year,Month,day,Week Ending,Geo Level,State,watershed_code,Commodity,Data Item,Domain,Domain Category,price,chicken,broken eggs,production eggs,chicken production,chicken died,Hatch chicken,chicken lay
0,SURVEY,2023,2,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,2.61,"50,031,000","181,566,000","8,225,300,000",NaN,NaN,"52,048,000","2,171"
1,SURVEY,2023,1,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,3.00,"47,370,000","188,508,000","8,225,300,000",NaN,NaN,"50,748,000","2,413"
2,SURVEY,2022,4,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,2.30,"53,971,000","190,571,000","8,225,300,000",NaN,NaN,"50,636,000","2,390"


In [86]:
print("Number of rows in dataset:", len(df))

Number of rows in dataset: 291


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Program             291 non-null    object 
 1   Year                291 non-null    int64  
 2   Month               291 non-null    int64  
 3   day                 291 non-null    int64  
 4   Week Ending         0 non-null      float64
 5   Geo Level           291 non-null    object 
 6   State               291 non-null    object 
 7   watershed_code      291 non-null    int64  
 8   Commodity           291 non-null    object 
 9   Data Item           291 non-null    object 
 10  Domain              291 non-null    object 
 11  Domain Category     291 non-null    object 
 12  price               291 non-null    float64
 13  chicken             135 non-null    object 
 14  broken eggs         122 non-null    object 
 15  production eggs     183 non-null    object 
 16  chicken 

 Add code here to get NULL counts per column 

In [88]:

df.isna().sum()

Program                 0
Year                    0
Month                   0
day                     0
Week Ending           291
Geo Level               0
State                   0
watershed_code          0
Commodity               0
Data Item               0
Domain                  0
Domain Category         0
price                   0
chicken               156
broken eggs           169
production eggs       108
chicken production    277
chicken died          277
Hatch chicken         157
chicken lay           108
dtype: int64

useful summary statistics

In [89]:
#data is missing rows and we trimmed it to 123 because most columns are empty after 123
sub_df =df[:123]
print("Number of rows in dataset:", len(sub_df))

Number of rows in dataset: 123


In [90]:

df.describe()

,Year,Month,day,Week Ending,watershed_code,price
count,291.000000,291.000000,291.0,0.0,291.0,291.000000
mean,2010.542955,6.484536,1.0,NaN,0.0,0.949529
std,7.014324,3.480329,0.0,NaN,0.0,0.487655
min,1998.000000,1.000000,1.0,NaN,0.0,0.446000
25%,2004.500000,3.000000,1.0,NaN,0.0,0.642500
50%,2011.000000,6.000000,1.0,NaN,0.0,0.831000
75%,2017.000000,9.500000,1.0,NaN,0.0,1.080000
max,2023.000000,12.000000,1.0,NaN,0.0,4.490000


In [91]:
df.dtypes

Program                object
Year                    int64
Month                   int64
day                     int64
Week Ending           float64
Geo Level              object
State                  object
watershed_code          int64
Commodity              object
Data Item              object
Domain                 object
Domain Category        object
price                 float64
chicken                object
broken eggs            object
production eggs        object
chicken production     object
chicken died           object
Hatch chicken          object
chicken lay            object
dtype: object

pull numeric features

In [92]:
df['chicken'] = df['chicken'].str.replace(',','')
df['chicken'] = pd.to_numeric(df['chicken'])

In [93]:
df.dtypes

Program                object
Year                    int64
Month                   int64
day                     int64
Week Ending           float64
Geo Level              object
State                  object
watershed_code          int64
Commodity              object
Data Item              object
Domain                 object
Domain Category        object
price                 float64
chicken               float64
broken eggs            object
production eggs        object
chicken production     object
chicken died           object
Hatch chicken          object
chicken lay            object
dtype: object

In [94]:
 df['broken eggs'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 291 entries, 0 to 290
Series name: broken eggs
Non-Null Count  Dtype 
--------------  ----- 
122 non-null    object
dtypes: object(1)
memory usage: 2.4+ KB


change data type to int and manipulate them.

In [95]:
df['broken eggs'] = df['broken eggs'].str.replace(',', '')
df['broken eggs'] = pd.to_numeric(df['broken eggs'])

In [96]:
df['production eggs'] = df['production eggs'].str.replace(',', '')
df['production eggs'] = pd.to_numeric(df['production eggs'])

In [97]:
df['chicken production'] = df['chicken production'].str.replace(',', '')
df['chicken production'] = pd.to_numeric(df['chicken production'])

In [98]:
df['chicken died'] = df['chicken died'].str.replace(',', '')
df['chicken died'] = pd.to_numeric(df['chicken died'])

In [99]:

df['Hatch chicken'] = df['Hatch chicken'].str.replace(',', '')
df['Hatch chicken'] = pd.to_numeric(df['Hatch chicken'])


In [100]:
df['chicken lay'] = df['chicken lay'].str.replace(',', '')
df['chicken lay'] = pd.to_numeric(df['chicken lay'])


When change all object to int now again check the info

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 0 to 290
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Program             291 non-null    object        
 1   Year                291 non-null    int64         
 2   Month               291 non-null    int64         
 3   day                 291 non-null    int64         
 4   Week Ending         0 non-null      float64       
 5   Geo Level           291 non-null    object        
 6   State               291 non-null    object        
 7   watershed_code      291 non-null    int64         
 8   Commodity           291 non-null    object        
 9   Data Item           291 non-null    object        
 10  Domain              291 non-null    object        
 11  Domain Category     291 non-null    object        
 12  price               291 non-null    float64       
 13  chicken             135 non-null    float64       

In [136]:
df['Year'] = df['Year'].astype('str')
df['Month'] = df['Month'].astype('str')
df['day'] = df['day'].astype('str')

In [137]:
df['date'] = df['Year']+ ' - '+ df['Month']+ ' - '+ df['day']

In [139]:
df['date']

0       2023 - 2 - 1
1       2023 - 1 - 1
2       2022 - 4 - 1
3       2022 - 8 - 1
4      2022 - 12 - 1
           ...      
286     1999 - 5 - 1
287    1999 - 11 - 1
288    1999 - 10 - 1
289     1999 - 9 - 1
290    1998 - 12 - 1
Name: date, Length: 291, dtype: object

In [140]:
df3 = pd.DataFrame()
df3['ds'] = pd.to_datetime(df['date'])
df3['y'] = df['price']
df3.head()

,ds,y
0,2023-02-01,2.61
1,2023-01-01,3.00
2,2022-04-01,2.30
3,2022-08-01,1.94
4,2022-12-01,4.49


In [144]:

plt.xlabel('ds')
plt.ylabel('y')
plt.plot(df3)

AttributeError: ignored

In [138]:
df.describe()

,Week Ending,watershed_code,price,chicken,broken eggs,production eggs,chicken production,chicken died,Hatch chicken,chicken lay
count,0.0,291.0,291.000000,1.350000e+02,1.220000e+02,1.830000e+02,1.400000e+01,1.400000e+01,1.340000e+02,183.000000
mean,NaN,0.0,0.949529,4.671931e+07,1.900354e+08,8.225300e+09,8.805714e+09,1.213872e+08,4.851732e+07,2309.278689
std,NaN,0.0,0.487655,5.473522e+06,1.419773e+07,0.000000e+00,2.683022e+08,2.224575e+07,5.929576e+06,87.751359
min,NaN,0.0,0.446000,3.417800e+07,1.485300e+08,8.225300e+09,8.463000e+09,9.959600e+07,3.596800e+07,2024.000000
25%,NaN,0.0,0.642500,4.217300e+07,1.818470e+08,8.225300e+09,8.564550e+09,1.043600e+08,4.378825e+07,2259.500000
50%,NaN,0.0,0.831000,4.707200e+07,1.905275e+08,8.225300e+09,8.732700e+09,1.076054e+08,4.833900e+07,2317.000000
75%,NaN,0.0,1.080000,5.028300e+07,1.984330e+08,8.225300e+09,9.031050e+09,1.373939e+08,5.206150e+07,2367.500000
max,NaN,0.0,4.490000,5.997000e+07,2.242760e+08,8.225300e+09,9.222100e+09,1.640673e+08,6.231900e+07,2465.000000


In [104]:
df.dtypes

Program                       object
Year                           int64
Month                          int64
day                            int64
Week Ending                  float64
Geo Level                     object
State                         object
watershed_code                 int64
Commodity                     object
Data Item                     object
Domain                        object
Domain Category               object
price                        float64
chicken                      float64
broken eggs                  float64
production eggs              float64
chicken production           float64
chicken died                 float64
Hatch chicken                float64
chicken lay                  float64
date                  datetime64[ns]
dtype: object

In [105]:

df_with_numeric_features = df.select_dtypes(exclude=['object', 'datetime64[ns]'])
print("Numeric features:")
for col in df_with_numeric_features:
    print(f"{col}")

Numeric features:
Year
Month
day
Week Ending
watershed_code
price
chicken
broken eggs
production eggs
chicken production
chicken died
Hatch chicken
chicken lay


pull non-numeric features

In [106]:

df_with_non_numeric_features = df.select_dtypes(include=['object'])
print("Non-numeric columns:")
for col in df_with_non_numeric_features:
    print(f"{col}")

Non-numeric columns:
Program
Geo Level
State
Commodity
Data Item
Domain
Domain Category


Remove duplicates
Never assume there are no duplicates in your data!

In [107]:
column_names = df.columns.tolist()
column_names

['Program',
 'Year',
 'Month',
 'day',
 'Week Ending',
 'Geo Level',
 'State',
 'watershed_code',
 'Commodity',
 'Data Item',
 'Domain',
 'Domain Category',
 'price',
 'chicken',
 'broken eggs',
 'production eggs',
 'chicken production',
 'chicken died',
 'Hatch chicken',
 'chicken lay',
 'date']

In [108]:
#remove duplicates rows

df.drop_duplicates(subset=column_names, keep='last', inplace=True)
print("Number of rows after removing duplicates", len(df))


Number of rows after removing duplicates 291


Drop null values from month column.


In [109]:
df.dropna(subset = ["date"], inplace=True)


In [125]:
df2 = pd.read_csv("/content/drive/MyDrive/The Eggcellent Predictor /Data/eggprice2.csv", parse_dates = ['Month'], index_col = ['Month'])
df2.head()


,Program,Year,day,Week Ending,Geo Level,State,watershed_code,Commodity,Data Item,Domain,Domain Category,price,chicken,broken eggs,production eggs,chicken production,chicken died,Hatch chicken,chicken lay
Month,,,,,,,,,,,,,,,,,,,
2,SURVEY,2023,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,2.61,"50,031,000","181,566,000","8,225,300,000",NaN,NaN,"52,048,000","2,171"
1,SURVEY,2023,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,3.00,"47,370,000","188,508,000","8,225,300,000",NaN,NaN,"50,748,000","2,413"
4,SURVEY,2022,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,2.30,"53,971,000","190,571,000","8,225,300,000",NaN,NaN,"50,636,000","2,390"
8,SURVEY,2022,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,1.94,"57,156,000","209,396,000","8,225,300,000",NaN,NaN,"55,991,000","2,449"
12,SURVEY,2022,1,NaN,NATIONAL,US TOTAL,0,EGGS,"EGGS - PRICE RECEIVED, MEASURED IN $ / DOZEN",TOTAL,NOT SPECIFIED,4.49,"49,852,000","181,674,000","8,225,300,000",NaN,NaN,"48,438,000","2,434"


SyntaxError: ignored

Describe Max and Min of the price 

In [ ]:
plt.hist(df.price,bins=10)
print(df.price.describe())

In [ ]:
df['Year'].value_counts()

Graph below shows that when chicken lay is high, the prices are low. (This would a good presentaion visual)

In [ ]:
price = df['price']
chicken = df['chicken lay']

# create a scatter plot
plt.scatter(chicken , price)
plt.xlabel('chicken lay')
plt.ylabel('Price')
plt.title('Price vs. Chicken lay')
plt.show()

check egg price data is stationery or not?

In [ ]:
from statsmodels.tsa.stattools import adfuller
x=df['price']
result=adfuller(x)
print("ADF Stataics ",result[0])
print("p-value",result[1])
print("critical values",result[5])
if result[1]<=0.05:
    print("fail to reject null hypothese h1 , it mean data is stationary")
else:
    print("Reject the null hypotheise , it mean data is not stationary")

This is stationary meaning that we can use it for the Arima Model. Stationary means? if p value is less than 0.5 

In [ ]:
#Now checking data is stationary or not using graphs.
df['price'].plot()

For ARIMA forcasting we need two parameter q and q that P is  parameter is an integer that confirms how many lagged series are going to be used to forecast periods ahead.and q is the number of lagged forecast error terms in the prediction equation.now we want to calculate P and q

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
plot_acf(df['price'].iloc[1:],lags=30)
plot_pacf(df['price'].iloc[1:],lags=30)


P=3 and q =2 

#Forecasted Data

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
model = sm.tsa.arima.ARIMA(df['price'], order=(1,1,2))
result = model.fit()
df['forecast_ARIMA'] = result.predict(start = 92, end= 114, dynamic= True)  
df[['price', 'forecast_ARIMA']].plot(figsize=(8, 5))


Check how SARIMA will perform on this seasonal data.

In [ ]:
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(df['price'],order=(2,1,2),seasonal_order=(2,1,2,6))
result=model.fit()

In [ ]:
df['forcast_SARIMA_1']=result.predict(start=99, end=112, dynamic=True)
df[['price','forcast_SARIMA_1']].plot(figsize=(20, 5))

SARIMA model is working well. because data is seasonal.

So, we are try to predict future forecasting using SARIMA model.



In [ ]:

import datetime
from dateutil.relativedelta import relativedelta
start = datetime.datetime.strptime("1969-07-01", "%Y-%m-%d")
date_list = [start + relativedelta(months=x) for x in range(0,12)]
future_prediction = pd.DataFrame(index=date_list, columns= df.columns)
future = pd.DataFrame(columns=df.columns) 
df= pd.concat([df, future])

**Predict future forecasting using SARIMA**

In [ ]:
df['future_prediction']=result.predict(start=113, end=130, dynamic=True)
df[['price','future_prediction']].plot(figsize=(10, 6))
plt.grid(True)

In [ ]:
df['future_prediction'] = result.predict(start=113, end=130, dynamic=True)
df[['price', 'future_prediction']].iloc[-20:].plot(figsize=(10, 6))

checking that te chicken lay is stationaryor not?

In [ ]:
x=sub_df['chicken lay']
result=adfuller(x)
print("ADF Stataics ",result[0])
print("p-value",result[1])
print("critical values",result[5])
if result[1]<=0.05:
    print("fail to reject null hypothese h1 , it mean data is stationary")
else:
    print("Reject the null hypotheise , it mean data is not stationary")

because p = 0.523 so it is not stationary.so change it to stationary.

In [ ]:
df['chicken lay first difference']=df['chicken lay']-df['chicken lay'].shift(1)

In [ ]:
df['chicken lay first difference'].plot()

it is now stationary because all around same number

# DO A CORRELATION MATRIC

In [ ]:
df.corr()

In [ ]:
corr_matrix = df.corr()
print(corr_matrix)

In [ ]:
sns.heatmap(corr_matrix, annot=True)
plt.show()

# The positive number shows good relationship and negative number shows week relation ship and zero shows no relationship.